In [1]:
import torch 
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from vocab import Vocab
from dataprocessor import DataProcessor

In [3]:
def get_data(filename):
    data = pd.read_csv(filename)
    return data['protein_sequence'].values , data['tm'].values 

trainx,trainy = get_data('./data/train_fixed.csv')
vocab = Vocab()
vocab.from_file()


dp = DataProcessor(vocab,trainx,trainy)
x,y = dp.sample(10)
print(x)
print(x.shape)
print(y)

tensor([[17,  5,  1,  ...,  0,  0,  0],
        [ 4,  6, 12,  ...,  0,  0,  0],
        [17,  2,  3,  ...,  0,  0,  0],
        ...,
        [17, 10,  4,  ...,  0,  0,  0],
        [ 5,  1,  4,  ...,  0,  0,  0],
        [17, 14, 10,  ...,  0,  0,  0]])
torch.Size([10, 575])
tensor([37.0000, 52.3000, 34.5000, 88.8000, 47.7000, 53.0000, 39.9000, 15.0000,
        44.3000, 49.8000])


In [ ]:
model = ConvProtein([21,32])

inputs = torch.randint(1,21,size=[13,60,60])

out = model(inputs)

print(out)